In [1]:
import jijzept as jz
import jijmodeling as jm

jijbenchを使わずにjijzeptとjijmodelingだけでTSPのベンチマークをとるパターン

In [2]:
# 問題の生成
problem = jm.Problem("travelling_salesman")
dist = jm.Placeholder("dist", dim=2)
N = jm.Placeholder("N")
x = jm.Binary("x", shape=(N, N))
i = jm.Element("i", N)
j = jm.Element("j", N)
t = jm.Element("t", N)
# Objective Funtion
sum_list = [t, i, j]
obj = jm.Sum(sum_list, dist[i, j] * x[t, i] * x[(t + 1) % N, j])
problem += obj
# const1: onehot for time
const1 = x[t, :]
problem += jm.Constraint(
    "onehot_time",
    const1 == 1,
    forall=[
        t,
    ],
)
# const2: onehot for location
const2 = x[:, i]
problem += jm.Constraint(
    "onehot_location",
    const2 == 1,
    forall=[
        i,
    ],
)

problem

In [3]:
# インスタンスデータの用意
import json

with open("tsp_sample_instance_data.json", "r") as f:
    ph_value = json.load(f)
    
ph_value

{'N': 36,
 'dist': [[100000000,
   26,
   82,
   65,
   102,
   100,
   147,
   134,
   69,
   117,
   125,
   38,
   40,
   13,
   38,
   31,
   22,
   103,
   143,
   94,
   104,
   123,
   140,
   98,
   58,
   38,
   30,
   67,
   120,
   149,
   100,
   141,
   93,
   162,
   62,
   66],
  [66,
   100000000,
   56,
   39,
   76,
   109,
   156,
   140,
   135,
   183,
   190,
   104,
   76,
   79,
   104,
   97,
   88,
   130,
   176,
   121,
   131,
   150,
   167,
   125,
   85,
   65,
   57,
   94,
   147,
   160,
   80,
   162,
   67,
   189,
   128,
   40],
  [43,
   57,
   100000000,
   16,
   20,
   53,
   100,
   84,
   107,
   155,
   168,
   81,
   53,
   56,
   81,
   74,
   65,
   146,
   186,
   137,
   147,
   166,
   183,
   141,
   101,
   81,
   73,
   110,
   163,
   164,
   102,
   166,
   71,
   205,
   105,
   62],
  [27,
   41,
   62,
   100000000,
   82,
   97,
   144,
   131,
   96,
   144,
   152,
   65,
   37,
   40,
   65,
   58,
   49,
   130,
   170,
 

In [6]:
problem.constraints

{'onehot_time': Cons(onehot_time)[Sum_{:x_1=0}^{N}(x[t,:x_1])=1],
 'onehot_location': Cons(onehot_location)[Sum_{:x_0=0}^{N}(x[:x_0,i])=1]}

In [28]:
import numpy as np

# num_sweepsの値を変えつつ、制約条件の係数の大きさで結果がどう変わるか調査したいという場合
sampler = jz.JijSASampler(config="/home/azureuser/.config/jijzept/config.toml")

num_reads_list = [10]
onehot_time_list = [0.1, 0.5, 1, 5, 10]
onehot_location_list = [0.1, 0.5, 1, 5, 10]

opt_value = ph_value["opt_value"]
pr = 0.99
metrics = {
    "num_reads": [],
    "onehot_time": [],
    "onehot_location": [],
    "execution_time": [],
    "success_probability": [],
    "feasible_rate": [],
    "residual_energy": [],
    "min_energy": [],
    "mean_eneagy": [],
    "tts_optimal": [],
    "tts_feasible": [],
    "tts_derived": []
}
for num_reads in num_reads_list:
    for onehot_time in onehot_time_list:
        for onehot_location in onehot_location_list:
            response = sampler.sample_model(
                problem,
                ph_value,
                {"onehot_time": onehot_time, "onehot_location": onehot_location},
                num_reads=num_reads,
            )
            decoded = problem.decode(response, ph_value, {})
            
            t = response.info["sampling_time"] / num_reads
            
            energy = decoded.energies
            energy_min = energy.min()
            energy_mean = energy.mean()
            residual_energy = (energy_mean - opt_value) / np.abs(opt_value)
            
            num_feasible = len(decoded.feasibles())
            feasible_rate = num_feasible / num_reads
            
            ps_optimal = (energy <= opt_value).sum() / num_reads
            ps_derived = (energy <= energy_min).sum() / num_reads
                
            tts_optimal = np.log(1 - pr) / np.log(1 - ps_optimal) * t
            tts_feasible = np.log(1 - pr) / np.log(1 - feasible_rate) * t
            tts_derived = np.log(1 - pr) / np.log(1 - ps_derived) * t
            
            metrics["num_reads"].append(num_reads)
            metrics["onehot_time"].append(onehot_time)
            metrics["onehot_location"].append(onehot_location)
            metrics["execution_time"].append(t)
            metrics["min_energy"].append(energy_min)
            metrics["mean_eneagy"].append(energy_mean)
            metrics["success_probability"].append(ps_optimal)
            metrics["feasible_rate"].append(feasible_rate)
            metrics["residual_energy"].append(residual_energy)
            metrics["tts_optimal"].append(tts_optimal)
            metrics["tts_feasible"].append(tts_feasible)
            metrics["tts_derived"].append(tts_derived)

uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is b74ff3cae15f4086ae9978ea398cea3a.
running...
uploading instance ...


/tmp/ipykernel_25265/4260298595.py:50: RuntimeWarning: divide by zero encountered in log
  tts_optimal = np.log(1 - pr) / np.log(1 - ps_optimal) * t
/tmp/ipykernel_25265/4260298595.py:51: RuntimeWarning: divide by zero encountered in log
  tts_feasible = np.log(1 - pr) / np.log(1 - feasible_rate) * t


submitting query ...
submitted to the queue.
Your solution_id is 1c66cbdc1bd0443d898e22611ad40cb2.
running...
uploading instance ...


/tmp/ipykernel_25265/4260298595.py:51: RuntimeWarning: divide by zero encountered in double_scalars
  tts_feasible = np.log(1 - pr) / np.log(1 - feasible_rate) * t


submitting query ...
submitted to the queue.
Your solution_id is bdfbff35bb314a7e978e5141c892f070.
running...
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is 637c9a62e4f3450a8d237ec6d2fef369.
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is e597a255e8cb4b4d8505a77ec7293675.
running...
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is df3181388f8f4872906135a329d15cbd.
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is c22becb6254841b3b6956c29a28e9371.
running...
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is a5500a5612264af99226bed416333a98.
running...
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is a3819669c2504afea5171b8bff566b73.
running...
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is 578665bdc43d43efbc1c10f389

In [29]:
import pandas as pd

metrics_df = pd.DataFrame(metrics)
metrics_df

,num_reads,onehot_time,onehot_location,execution_time,success_probability,feasible_rate,residual_energy,min_energy,mean_eneagy,tts_optimal,tts_feasible,tts_derived
0,10,0.1,0.1,84676.700109,1.0,1.0,-1.002444,-3.599953,-3.599951,0.0,0.000000e+00,3.701108e+06
1,10,0.1,0.5,82627.976895,1.0,0.0,-1.007047,-10.499956,-10.379950,0.0,-inf,3.611561e+06
2,10,0.1,1.0,81702.375691,1.0,0.0,-1.012919,-19.299952,-19.029952,0.0,-inf,3.571104e+06
3,10,0.1,5.0,92308.437452,1.0,0.0,-1.061426,-90.999947,-90.479953,0.0,-inf,4.034681e+06
4,10,0.1,10.0,135368.595086,1.0,0.0,-1.122342,-180.899952,-180.209951,0.0,-inf,5.916784e+06
5,10,0.5,0.1,80881.679500,1.0,0.0,-1.006999,-10.599955,-10.309953,0.0,-inf,3.535232e+06
6,10,0.5,0.5,95157.854632,1.0,1.0,-1.012220,-17.999958,-17.999953,0.0,0.000000e+00,4.159225e+06
7,10,0.5,1.0,94982.587732,1.0,0.3,-1.018092,-26.999957,-26.649952,0.0,1.226357e+06,4.151565e+06
8,10,0.5,5.0,93085.765652,1.0,0.0,-1.064868,-95.999953,-95.549950,0.0,-inf,4.068657e+06
9,10,0.5,10.0,147398.026474,1.0,0.0,-1.124779,-185.499949,-183.799950,0.0,-inf,6.442575e+06


同様のベンチマークをJijBenchmarkを使うとどうなるか？

In [1]:
import jijbench as jb

num_reads_list = [10]
onehot_time_list = [0.1, 0.5, 1, 5, 10]
onehot_location_list = [0.1, 0.5, 1, 5, 10]
multipliers_list = [
    {"onehot_time": onehot_time, "onehot_location": onehot_location}
    for onehot_time in onehot_time_list
    for onehot_location in onehot_location_list
]

bench = jb.Benchmark(
    params={"num_reads": num_reads_list, "multipliers": multipliers_list},
    solvers="JijSASampler",
    targets="TSP",
    jijzept_config="/home/azureuser/.config/jijzept/config.toml"
)
# sync=Falseにするとベンチマークの時間を短縮できます。
bench.run(sync=False)
# TTSなど典型的な指標を計算できます。
bench.evaluate(pr=0.99)

uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is ae173478204b44daa8752902b8e231ad.
You can access the solution by
>>> a = XXXSampler(config=...)
>>> a.get_result("ae173478204b44daa8752902b8e231ad")
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is 59088b012e0c4827909619d09798776b.
You can access the solution by
>>> a = XXXSampler(config=...)
>>> a.get_result("59088b012e0c4827909619d09798776b")
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is 87b1a49002414f06a9d92b5c8116068b.
You can access the solution by
>>> a = XXXSampler(config=...)
>>> a.get_result("87b1a49002414f06a9d92b5c8116068b")
uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is d4d93c3354b841f9b755990b9bd93cff.
You can access the solution by
>>> a = XXXSampler(config=...)
>>> a.get_result("d4d93c3354b841f9b755990b9bd93cff")
uploading instance ...
submitting query ...
submitte

/home/azureuser/JijBenchmark/jijbench/experiment/experiment.py:81: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self._table.data.loc[self._table.current_index] = np.nan
/home/azureuser/JijBenchmark/jijbench/experiment/experiment.py:81: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self._table.data.loc[self._table.current_index] = np.nan
/home/azureuser/JijBenchmark/jijbench/experiment/experiment.py:81: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to ob

,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,1.0,1.0,-1.002444,0.0,0.000000e+00,4.191062e+06
1,1.0,0.0,-1.007026,0.0,-inf,3.603286e+06
2,1.0,0.0,-1.012946,0.0,-inf,4.088177e+06
3,1.0,0.0,-1.061439,0.0,-inf,3.673113e+06
4,1.0,0.0,-1.122322,0.0,-inf,6.016795e+06
5,1.0,0.0,-1.006979,0.0,-inf,3.586428e+06
6,1.0,1.0,-1.012220,0.0,0.000000e+00,4.218451e+06
7,1.0,0.6,-1.018194,0.0,4.261224e+05,3.705866e+06
8,1.0,0.0,-1.064257,0.0,-inf,4.055827e+06
9,1.0,0.0,-1.124915,0.0,-inf,6.356804e+06


In [2]:
# 結果は全てBenchmark.table(pandas.DataFrame)に保存される。
bench.table

,benchmark_id,experiment_id,run_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,onehot_location_violations,onehot_location_violation_min,onehot_location_violation_mean,onehot_location_violation_std,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,7fd680e9-17e9-405c-876e-93f53c286d67,2022-03-31 23:17:00.127392,"[-3.5999515300015794, -3.5999515500015793, -3....",-3.599959,-3.599953,0.000003,"[4847.0, 4845.0, 4087.0, 4891.0, 4371.0, 4845....",4087.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,1.0,-1.002444,0.0,0.000000e+00,4.191062e+06
1,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,fb804c9f-c154-4bb9-adf3-ff1efa954459,2022-03-31 23:17:18.647205,"[-10.19995399999093, -10.39995516999093, -10.2...",-10.599955,-10.349952,0.111805,"[4600.0, 4483.0, 4937.0, 4808.0, 4894.0, 4555....",4453.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,0.0,-1.007026,0.0,-inf,3.603286e+06
2,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,b7fab8eb-8068-43f0-b9f4-51d61ca3237a,2022-03-31 23:17:36.922251,"[-18.99995295999457, -19.09995151999457, -19.0...",-19.299953,-19.069953,0.173492,"[4704.0, 4848.0, 4406.0, 5123.0, 5457.0, 4415....",4305.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,0.0,-1.012946,0.0,-inf,4.088177e+06
3,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,96768df9-00c9-4d68-9a7c-3c16d3611742,2022-03-31 23:17:55.435270,"[-90.29995588001809, -90.09994141001809, -90.0...",-91.099944,-90.499951,0.357771,"[4412.0, 5859.0, 4859.0, 4470.0, 4907.0, 5146....",4242.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,0.0,-1.061439,0.0,-inf,3.673113e+06
4,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,0a4e5af5-28b4-4a8a-8799-c883353ae205,2022-03-31 23:18:13.826104,"[-179.79994511001496, -179.69995458001497, -18...",-180.899961,-180.17995,0.370948,"[5489.0, 4542.0, 4794.0, 3875.0, 5757.0, 6288....",3745.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,0.0,-1.122322,0.0,-inf,6.016795e+06
5,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,a6e3ef96-df45-45da-b228-40a037e3af90,2022-03-31 23:18:32.564403,"[-10.199949029989112, -10.399945949989112, -10...",-10.399954,-10.279951,0.116618,"[5097.0, 5405.0, 5213.0, 4778.0, 4632.0, 4616....",4476.0,...,"[12.0, 8.0, 8.0, 14.0, 14.0, 8.0, 10.0, 8.0, 1...",8.0,10.4,2.332381,1.0,0.0,-1.006979,0.0,-inf,3.586428e+06
6,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,28c9ef8a-2182-4b38-abb4-722113463ef1,2022-03-31 23:18:50.931410,"[-17.99995531, -17.99995553, -17.99995424, -17...",-17.999956,-17.999953,0.000002,"[4469.0, 4447.0, 4576.0, 4997.0, 5012.0, 4614....",4447.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,1.0,-1.012220,0.0,0.000000e+00,4.218451e+06
7,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,d508efcd-23ca-43ef-bfc3-fdfba12e4ab9,2022-03-31 23:19:09.415162,"[-26.99995290998545, -26.99995007998545, -26.4...",-26.999958,-26.799953,0.244948,"[4709.0, 4992.0, 4546.0, 4975.0, 4832.0, 4661....",4217.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,0.6,-1.018194,0.0,4.261224e+05,3.705866e+06
8,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,4a0b1de9-dc53-444a-842d-2c4653c63976,2022-03-31 23:19:27.973188,"[-95.99995190003638, -94.49995300003638, -93.9...",-96.499953,-94.649951,1.265899,"[4810.0, 4700.0, 4354.0, 6150.0, 5069.0, 4109....",4109.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000,1.0,0.0,-1.064257,0.0,-inf,4.055827e+06
9,6de723fd-6b41-4ed1-927d-85261e9aff98,8ee54693-7285-4164-9a84-f4c055ac3eeb,c1f837b6-a1d8-4785-a1f7-adbbc

In [6]:
# bench.tableの一部だけ表示
columns = [
        "num_reads",
        "multipliers",
        "execution_time",
        "success_probability",
        "feasible_rate",
        "residual_energy",
        "energy_min",
        "energy_mean",
        "TTS(optimal)",
        "TTS(feasible)",
        "TTS(derived)",
]
bench.table[columns]

,num_reads,multipliers,execution_time,success_probability,feasible_rate,residual_energy,energy_min,energy_mean,TTS(optimal),TTS(feasible),TTS(derived)
0,10,"{'onehot_time': 0.1, 'onehot_location': 0.1}",95886.230469,1.0,1.0,-1.002444,-3.599959,-3.599953,0.0,0.000000e+00,4.191062e+06
1,10,"{'onehot_time': 0.1, 'onehot_location': 0.5}",82438.655198,1.0,0.0,-1.007026,-10.599955,-10.349952,0.0,-inf,3.603286e+06
2,10,"{'onehot_time': 0.1, 'onehot_location': 1}",93532.365561,1.0,0.0,-1.012946,-19.299953,-19.069953,0.0,-inf,4.088177e+06
3,10,"{'onehot_time': 0.1, 'onehot_location': 5}",84036.210598,1.0,0.0,-1.061439,-91.099944,-90.499951,0.0,-inf,3.673113e+06
4,10,"{'onehot_time': 0.1, 'onehot_location': 10}",137656.714395,1.0,0.0,-1.122322,-180.899961,-180.17995,0.0,-inf,6.016795e+06
5,10,"{'onehot_time': 0.5, 'onehot_location': 0.1}",82052.978012,1.0,0.0,-1.006979,-10.399954,-10.279951,0.0,-inf,3.586428e+06
6,10,"{'onehot_time': 0.5, 'onehot_location': 0.5}",96512.869559,1.0,1.0,-1.012220,-17.999956,-17.999953,0.0,0.000000e+00,4.218451e+06
7,10,"{'onehot_time': 0.5, 'onehot_location': 1}",84785.56741,1.0,0.6,-1.018194,-26.999958,-26.799953,0.0,4.261224e+05,3.705866e+06
8,10,"{'onehot_time': 0.5, 'onehot_location': 5}",92792.224884,1.0,0.0,-1.064257,-96.499953,-94.649951,0.0,-inf,4.055827e+06
9,10,"{'onehot_time': 0.5, 'onehot_location': 10}",145435.697958,1.0,0.0,-1.124915,-185.999946,-183.999956,0.0,-inf,6.356804e+06
